In [54]:
import pandas as pd

import os
from dotenv import load_dotenv, dotenv_values, set_key
# import pathlib

from os.path import join, dirname

path = 'D:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\'
dotenv_path = join(path, '.env')

load_dotenv(dotenv_path)

CR_Token = os.getenv(f'CR_Token')
adm_token = os.getenv(f'ADM_TOKEN')
ASTRA_DB_SECURE_BUNDLE_PATH = os.getenv(f'ASTRA_DB_SECURE_BUNDLE_PATH')
DB_ID = os.getenv(f'DB_ID')

In [55]:
from astrapy import DataAPIClient

# Initialize the client
# client = DataAPIClient(Token['token'])
client = DataAPIClient(adm_token)


In [56]:
# db = client.get_database(
#     id=DB_ID
# )

# print(f"Connected to Astra DB: {db.list_collection_names()}")

C:\Users\hatem.kamal\AppData\Local\Temp\ipykernel_45884\2774269626.py:1: DeprecatedWarning: Passing an `id` parameter is deprecated as of 1.5.1 and will be removed in 2.0.0. Please switch to using the API Endpoint.
  db = client.get_database(


Connected to Astra DB: []


In [57]:
# db.command("""
#         CREATE KEYSPACE IF NOT EXISTS udacity 
#         WITH REPLICATION = 
#         { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
#         """)
# should be done in astra portal

In [58]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# with open(ASTRA_TOKEN_PATH, "r") as f:
#     creds = json.load(f)
    # ASTRA_DB_APPLICATION_TOKEN = creds["token"]

ASTRA_DB_APPLICATION_TOKEN = adm_token
try: 
    cluster = Cluster(
        cloud={
            "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
        },
        auth_provider=PlainTextAuthProvider(
            "token",
            ASTRA_DB_APPLICATION_TOKEN,
        ),
    )

    session = cluster.connect()


    session = cluster.connect()
except Exception as e:
    print(e)

In [59]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };"""
)

except Exception as e:
    print(e)

Error from server: code=2100 [Unauthorized] message="Missing correct permission on udacity."


In [60]:
row = session.execute("select release_version from system.local").one()
if row:
    print(row[0])
else:
    print("An error occurred.")

4.0.11-7cde36df13c4


In [61]:
try:
    session.set_keyspace('project_keyspace')
except Exception as e:
    print(e)

### Let's imagine we would like to start creating a Music Library of albums. 

### We want to ask 2 questions of our data
#### 1. Give me every album in my music library that was released in a given year
`select * from music_library WHERE YEAR=1970`
#### 2. Give me every album in my music library that was created by a given artist  
`select * from artist_library WHERE artist_name="The Beatles"`


### Because I want to do two different queries, I am going to do need different tables that partition the data differently. 
* My music library table will be by year that will become my partition key, and artist name will be my clustering column to make each Primary Key unique. 
* My album library table will be by artist name that will be my partition key, and year will be my clustering column to make each Primary Key unique. More on Primary keys in the next lesson and demo. 

`Table Name: music_library
column 1: Year
column 2: Artist Name
column 3: Album Name
PRIMARY KEY(year, artist name)`


` Table Name: album_library 
column 1: Artist Name
column 2: Year
column 3: Album Name
PRIMARY KEY (artist name, year)`


In [62]:
create_query = "CREATE TABLE IF NOT EXISTS  "
query = create_query + "music_library (year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS  "
query = create_query + "album_library (year int, artist_name text, album_name text, PRIMARY KEY (artist_name,  year))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [63]:
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + " VALUES (%s, %s, %s)"

query1 = "INSERT INTO album_library (artist_name, year, album_name)"
query1 = query1 + " VALUES (%s, %s, %s)"

try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Who", "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1966, "The Monkees", "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1970, "The Carpenters", "Close To You"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Beatles", 1970, "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Beatles", 1965, "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Who", 1965, "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Monkees", 1966, "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Carpenters", 1970, "Close To You"))
except Exception as e:
    print(e)

In [64]:
query = "select * from music_library WHERE YEAR=1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.artist_name, row.album_name,)

1970 The Beatles Let it Be
1970 The Carpenters Close To You


In [65]:
query = "select * from album_library WHERE ARTIST_NAME='The Beatles'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.year, row.album_name)

The Beatles 1965 Rubber Soul
The Beatles 1970 Let it Be


In [66]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table album_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

errors={'Connection defunct by heartbeat': 'Client request timeout. See Session.execute[_async](timeout)'}, last_host=d1d2952f-cedc-4554-acc9-3713e6599c87-us-east-2.db.astra.datastax.com:29042:9e3a5bee-3d95-3bf7-90f5-09bd2177324b


In [67]:
session.shutdown()
cluster.shutdown()